In [1]:
from bs4 import BeautifulSoup
from cltk.stem.lemma import LemmaReplacer
from cltk.tokenize.word import WordTokenizer
from collections import defaultdict
import json
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
import os
import re
import string
from cltk.tokenize.sentence import TokenizeSentence
from nltk.tokenize import sent_tokenize
import bs4

def make_list(soup):
    l = []
    for n in soup:
        try:
            l.append(int(n['n']))
        except:
            l.append(-1)  
    return remove_duplicates(l)


def remove_duplicates(l):
    i = 0
    while i+1 < len(l):
        if l[i] == l[i+1]:
            l[i+1] += 1
        if l[i] > l[i+1] and l[i+1] != -1:
            l[i] = l[i+1]-1
        i += 1
    return l

def match(large,small):
    l = make_list(large)
    s = make_list(small)
    li = 0
    si = 0
    groups = []
    #print(l,s)
    while si < len(s) and li < len(l):
        if s[si] == -1:
            si += 1
            continue
        if l[li] == -1:
            li += 1
            continue
        lgroup = []
        sgroup = []
        if l[li] == s[si]:
            lgroup.append(li)
            sgroup.append(si)
              
        if si+1 < len(s) and li+1 < len(l): 
            if s[si+1] == -1:
                si += 1
            
            if l[li+1] == -1:
                li += 1
            
            
            if l[li+1] != s[si+1]:
                while l[li+1] != s[si+1]:
                    if si+1 < len(s) and li+1 < len(l):
                        if l[li+1] > s[si+1]:
                            sgroup.append(si+1)
                            si += 1
                            if s[si] == -1:
                                if si+1 < len(s):
                                    si += 1
                                else:
                                    pass
                        else:
                            lgroup.append(li+1)
                            li += 1
                            if l[li] == -1:
                                if li+1 < len(l):
                                    li += 1
                                else:
                                    pass
                    else:
                        break
                si += 1
                li += 1
                groups.append([sgroup,lgroup])
            else:
                si += 1
                li += 1
                groups.append([sgroup,lgroup]) 
        else:
            break        
    
    groups.append([sgroup,lgroup])
    return groups
        

def align_soups(m,small,large):
    s_alignments = []
    l_alignments = []
    for g in m:
        s = []
        l = []
        for i in g[0]:
            s.append(small[i])
        for i in g[1]:
            l.append(large[i])
        s_alignments.append(s)
        l_alignments.append(l)   
    return s_alignments,l_alignments


def get_pairs(latin_soup, eng_soup, tag, typename, typos):
    """ Given two soups, returns soups of equal size, with each element corresponding to the same
        element in the other soup.
        Lists may contain further lists of soups which are aligned with one another, they are combined into one list
        in the first step of this function.
    """
    lat_sect,eng_sect = [], []
    
    for i in range(len(latin_soup)):
        lat_sect += latin_soup[i].findAll(tag, {typename: typos})
    for i in range(len(eng_soup)):
        eng_sect += eng_soup[i].findAll(tag, {typename: typos})

    if len(lat_sect) > len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].parent['n'])
            m = match(lat_sect,eng_sect)
            eng_alignments,lat_alignments = align_soups(m,eng_sect,lat_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    elif len(lat_sect) < len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].name)
            m = match(eng_sect,lat_sect)
            lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    else:
        if eng_sect[0]['n'] == 'intro':
            try:
                #print(latin_soup[0]['n'],latin_soup[0].name)
                m = match(eng_sect,lat_sect)
                lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
                return lat_alignments,eng_alignments
            except:
                print("error with {}, {}".format(tag,typos))
        else:
            
            return lat_sect, eng_sect
    



def align_pars(lsentences,esentences):
    
    if lsentences[-1] == '':
        lsentences = lsentences[:-1]
    if esentences[-1] == '':
        esentences = esentences[:-1]
        
    if len(lsentences) > len(esentences):
        alignments = align(lsentences,esentences)
        target = esentences
        source = lsentences
    elif len(lsentences) < len(esentences):
        alignments = align(esentences,lsentences)
        target = lsentences
        source = esentences
    else:
        return zip(lsentences,esentences)
    
    sentence_tuples = []
    for k in alignments.keys():
        sentence_tuples.append((target[k], [source[j] for j in alignments[k]]))
        
    return sentence_tuples
          



def clean_lemma(word):
    try:
        word = re.sub('[0-9]+', '', word)
        word = re.sub('[-_]+', '', word)
    except:
        pass

    return word


def get_lemmata_defs(latin):
    words = lemmatizer.lemmatize(latin)
    ldefs = []
    unmatched = []

    for word in words:
        word = clean_lemma(word)
        try:
            ldefs.append(defs[word]['definition'].split(','))
        except:
            unmatched.append(word)
    return ldefs,unmatched


def get_defs(ldefs):
    lldefs=[]
    for l in ldefs:
        lldefs += t(l[0])
    return lldefs


def get_match_count(english,latin):
    
    ldefs,unmatched = get_lemmata_defs(latin)
    ldefs = get_defs(ldefs)
    count = 0
    for word in t(english):
        if word in string.punctuation:
            continue
        if word in ldefs and word not in stop:
            count += 1
        if word in unmatched:
            count += 1
    return count


def align(source,target):
    """ Takes a paragraph of sentences, source, and maps each sentence to a sentence in the 
        target paragraph. 
        
        Returns a dictionary with the target sentence indices as keys, and the sentences in the source paragraph
        which correspond to them as values.
    """
    tsize = len(target)
    ssize = len(source)
    alignments = defaultdict(list)
    alignments[0] = [0]
    i = 1
    j = 1
    while tsize < ssize and i < len(source) and j < tsize:
        max_count = get_match_count(source[i],target[j])
        max_i = j
        if j >= tsize-1:
            rge = [j-1]
        else:
            rge = [j-1,j+1]
        for r in rge: 
            c = get_match_count(source[i],target[r])
            if c > max_count:
                max_count = c
                max_i = r

        if max_i != j:
            ssize -= 1
        else:
            j += 1

        alignments[max_i].append(i)
        i += 1
        
  
    while j < tsize:
        alignments[j].append(i)
        j += 1
        i += 1
    alignments[j-1].extend([k for k in range(i,len(source))])
    
    return alignments

def get_sec_alignments(lpar,epar):
    if len(lpar) > len(epar):
        if len(epar) > 1:
            alignments = align(lpar,epar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(lpar[v])
                aligned.append((group,epar[k]))
        else:
            try:
                aligned = [('. '.join(lpar),epar[0])] 
            except:
                print('error, {}'.format(epar))
                aligned = list(zip(lpar,epar))
    elif len(epar) > len(lpar):
        if len(lpar) > 1:
            alignments = align(epar,lpar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(epar[v])
                aligned.append((lpar[k],group))
        else:
            try:
                aligned = [(lpar[0],'. '.join(epar))]
            except:
                print('error, {}'.format(lpar))
                aligned = list(zip(lpar,epar))
    else:
        aligned = list(zip(lpar,epar))
        
    return aligned

def get_sentences(chap_soup, sect_soup):
    t = bs4.element.Tag
    sect = []
    sentences = []
    i = 0
    c = 0
    children = list(chap_soup.children)
    soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
    c += 1
    while type(soup) != t:
        soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
        c += 1
    i += 1
    while c < len(children):
        print(sentences)
        while soup.next_sibling != None:
            soup = soup.next_sibling
            if soup.name == 'milestone':
                try:
                    if int(soup['n']) == int(sect_soup[i]['n']):
                        i += 1
                        sentences.append(' '.join(sect))
                        sect = []
                except:
                    soup = soup.next_sibling
                    i += 1
            if soup.name in ['corr', 'name','hi','quote','reg', 'l', 'lg', 'placeName']:
                    sect.append(soup.text.strip())
            elif soup.name == None:
                    sect.append(soup.strip())
                    
        sentences.append(' '.join(sect))
        sect = []
        try:
            soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
            c += 1
            while type(soup) != t and c < len(children):
                soup = children[c].find('milestone')
                c += 1
            i += 1
        except:
            continue
    sentences.append(' '.join(sect))
    sect = []
    return sentences

In [2]:
t = wordpunct_tokenize
stop = set(stopwords.words('english'))

f = open('../lemmas.json')
defs = json.loads(f.read())

lemmatizer = LemmaReplacer('latin')
tokenizer = WordTokenizer('latin')
s_tokenizer = TokenizeSentence('latin')

In [9]:
f = open('../../Latin/Vitruvius/opensource/vitruv_lat.xml')
ltext = f.read()
f.close()
f = open('../../Latin/Vitruvius/opensource/vitruv_eng.xml')
etext = f.read()
f.close()

lat_sections = re.split(r'<div1 type="book" n="\d*">', ltext)[1:]
eng_sections = re.split(r'<div1 type="book" n="\d*">', etext)[1:]

In [10]:
len(lat_sections)

10

In [11]:
len(eng_sections)

10

In [24]:
sects = list(zip(lat_sections,eng_sections))

In [25]:
lchaps = []
echaps = []

for sect in sects:
    lchaps.append(re.split(r'<div2 type="chapter" n=".+">', sect[0])[1:])
    echaps.append(re.split(r'<div2 type="chapter" n=".+">', sect[1])[1:])

In [61]:
lchaps[0][2]

'\n<div3 type="section" n="1"><p>Architectura autem constat ex ordinatione, quae graece\n<foreign lang="greek">ta/cis</foreign> dicitur, et ex dispositione, hanc autem Graeci <foreign lang="greek">dia/qesin</foreign> vocitant, et eurythmia et symmetria et decore et\ndistributione, quae graece <foreign lang="greek">oi)konomi/a</foreign> dicitur.\n</p></div3>\n<div3 type="section" n="2"><p>Ordinatio est [modica] membrorum operis [commoditas]\nseparatim universeque proportionis ad symmetriam comparatio. haec componitur ex quantitate, quae graece\n<foreign lang="greek">poso/ths</foreign> dicitur. quantitas autem est modulorum ex ipsius\noperis e singulisque membrorum partibus sumptio universi operis conveniens effectui.</p>\n<p>Dispositio autem est rerum apta conlocatio elegansque\ncompositionibus effectus operis cum qualitate. species dispositionis, quae graece dicuntur <foreign lang="greek">i)de/ai</foreign>, sunt hae: ichnographia, orthographia, scaenographia. ichnographia est\ncircini 

In [28]:
for i in range(len(lchaps)):
    print(len(lchaps[i]),len(echaps[i]))

8 8
11 11
6 6
10 10
13 13
9 9
15 15
7 7
9 9
17 17


In [22]:
sects = list(zip(lchaps,echaps)) 

In [69]:
lsects = []
esects = []

for b in range(len(lchaps)):
    for c in range(len(lchaps[b])):
        lsp = re.split(r'<div3 type="section" n="\d+">', lchaps[b][c])
        esp = re.split(r'<div3 type="section" n="\d+">', echaps[b][c])
        while '\n' in lsp:
            lsp.remove('\n')
        while '\n' in esp:
            esp.remove('\n')
        if '<head>' in esp[0]:
            esp = esp[1:]
        if '<head>' in lsp[0]:
            lsp = lsp[1:]
        lsects.append(lsp)
        esects.append(esp)

In [70]:
lsects[0]

['<p>Cum divina tua mens et numen, imperator Caesar, imperio potiretur orbis terrarum invictaque virtute cunctis\nhostibus stratis triumpho victoriaque tua cives gloriarentur\net gentes omnes subactae tuum spectarent nutum populusque Romanus et senatus liberatus timore amplissimis tuis\ncogitationibus consiliisque gubernaretur, non audebam,\ntantis occupationibus, de architectura scripta et magnis\ncogitationibus explicata edere, metuens, ne non apto tempore interpellans subirem tui animi offensionem.\n</p></div3>\n',
 '<p>Cum vero attenderem te non solum de vita communi omnium curam\npublicaeque rei constitutione habere sed etiam de opportunitate publicorum aedificiorum, ut civitas per te non\nsolum provinciis esset aucta, verum etiam ut maiestas\nimperii publicorum aedificiorum egregias haberet auctoritates, non putavi praetermittendum, quin primo quoque\ntempore de his rebus ea tibi ederem, ideo quod primum\nparenti tuo [de eo] fueram notus et eius virtutis studiosus.\ncum autem con

In [71]:
for i in range(len(lsects)):
    print(len(lsects[i]),len(esects[i]))

3 3
18 17
9 9
2 2
12 12
8 8
13 13
2 2
6 5
9 9
2 2
4 4
3 3
3 3
6 6
5 5
20 20
17 17
3 3
4 4
9 9
8 8
13 13
5 5
15 15
2 1
12 12
6 6
10 9
4 4
2 1
6 6
5 5
7 7
1 1
5 5
9 10
2 2
8 8
9 9
8 8
9 9
2 2
2 2
9 9
5 5
4 4
7 7
7 7
12 12
5 5
11 11
2 2
3 3
7 7
7 7
10 10
18 18
7 7
2 2
11 11
5 5
8 8
1 1
5 5
4 4
6 6
4 4
2 2
2 2
3 3
3 3
4 4
7 7
9 9
28 28
2 2
3 3
15 14
18 18
16 16
4 4
3 3
6 6
4 4
3 3
7 7
15 15
4 4
6 6
15 15
9 9
4 4
2 2
4 4
5 5
6 6
7 7
6 6
9 9
2 2
8 8
3 3
7 7
12 12


In [73]:
eng_sects = esects
lat_sects = lsects

In [74]:
lat_text = []
eng_text = []

for s in range(len(eng_sects)):
    if len(eng_sects[s]) != len(lat_sects[s]):
        continue
        
    for c in range(len(eng_sects[s])):
        es = eng_sects[s][c]
        if re.match(r'</.>',es):
            es = re.split(r'</.>',es,maxsplit=1)[1]
        soup = BeautifulSoup(es,'lxml')
        if soup.find('note'):
            es = re.sub(r'<note[^>.]*>', '<note>', es,flags=re.DOTALL)
            sp = re.split(r'<note>',es)
            es = sp[0]
            for n in sp[1:]:
                n = re.sub(r'.*</note>', " ", n,flags=re.DOTALL)
                es += n
            soup = BeautifulSoup(es,'lxml')
            eng_text.append(soup.text)
        else:
            eng_text.append(soup.text)

        ls = lat_sects[s][c]
        if re.match(r'</.>',ls):
            el = re.split(r'</.>',ls,maxsplit=1)[1]
        soup = BeautifulSoup(ls,'lxml')
        if soup.find('note'):
            ls = re.sub(r'<note[^>.]*>', '<note>', ls,flags=re.DOTALL)
            sp = re.split(r'<note>',ls)
            ls = sp[0]
            for n in sp[1:]:
                n = re.sub(r'.*</note>', " ", n,flags=re.DOTALL)
                ls += n
            soup = BeautifulSoup(ls,'lxml')
            lat_text.append(soup.text)
        else:
            lat_text.append(soup.text)

In [78]:
alignments = []

for c in range(len(eng_text)):
    es = ' '.join(eng_text[c].replace('\n',' ').split())
    ls = lat_text[c].replace('\n', ' ')

    ls = re.sub(r'\. \. \.','',ls)
    es = re.sub(r'\. \. \.','',es)
    ls = re.sub('[:;,]','',ls)
    es = re.sub('[:;,]','',es)
    es = re.sub('\d+\.','',es)

    epar = sent_tokenize(es)
    lpar = s_tokenizer.tokenize_sentences(ls)
    alignments.append(get_sec_alignments(lpar,epar))

error, []
error, []


In [79]:
alignments[1]

[('Cum vero attenderem te non solum de vita communi omnium curam publicaeque rei constitutione habere sed etiam de opportunitate publicorum aedificiorum ut civitas per te non solum provinciis esset aucta verum etiam ut maiestas imperii publicorum aedificiorum egregias haberet auctoritates non putavi praetermittendum quin primo quoque tempore de his rebus ea tibi ederem ideo quod primum parenti tuo [de eo] fueram notus et eius virtutis studiosus.',
  [' But when I saw that you were giving your attention not only to the welfare of society in general and to the establishment of public order but also to the providing of public buildings intended for utilitarian purposes so that not only should the State have been enriched with provinces by your means but that the greatness of its power might likewise be attended with distinguished authority in its public buildings I thought that I ought to take the first opportunity to lay before you my writings on this theme.']),
 ('cum autem concilium ca

In [80]:
count = 0
for a in alignments:
        count += len(a)

In [81]:
count

2208

In [82]:
f = open('../aligned_sentences/vitruvius_sentences.json','w') 
j = json.dumps(alignments)
f.write(j)
f.close()